# Correlated inputs

## Implementation of Xu: https://doi.org/10.1016/j.ress.2007.06.003

In [100]:
import numpy as np
import numpy.random as rand
import matplotlib.pyplot as plt
from scipy import stats

### Test their method on the first example

### 1. Model: $y = 2x_1 + 3x_2$

In [101]:
loc, scale = [0,0], [1,1]
r = 0.7 #correlation ratio is known

cov_x1   = scale[0]**2
cov_x2   = scale[1]**2
cov_x1x2 = r*scale[0]*scale[1]
cov = [[cov_x1,cov_x1x2],[cov_x1x2,cov_x2]]
         
N = 1000000 #Number of samples

x = rand.multivariate_normal(loc,cov,N)

x1,x2 = x[:,0],x[:,1]
y = 2*x1+3*x2

loc_y = np.mean(y)

### Analytical values

In [102]:
#Analytical values for variances

Vt_1 = 12*cov_x1x2 + 4*cov_x1 + r**2*9*cov_x2 #Total variations of x1
Vu_1 = (1-r**2)*4*cov_x1 #Uncorrelated variation of x1
Vc_1 = 12*cov_x1x2 + r**2*4*cov_x1 + r**2*9*cov_x2 #Correlated variation of x1

Vt_2 = 12*cov_x1x2 + r**2*4*cov_x1 + 9*cov_x2
Vu_2 = (1-r**2)*9*cov_x2
Vc_2 = Vc_1

### Regression-based method

In [103]:
theta_x1 = stats.linregress(x1,y)
theta_x2 = stats.linregress(x2,y)

#Predicted values (taken from regression)
y1_p = theta_x1.intercept + theta_x1.slope*x1
y2_p = theta_x2.intercept + theta_x2.slope*x2

#Total variations
Vt_1_p = sum((y1_p-loc_y)**2)/(N-1)
Vt_2_p = sum((y2_p-loc_y)**2)/(N-1)

#Uncorrelated variations
nu_1 = stats.linregress(x2,x1) #predicting values of x1 based on x2
nu_2 = stats.linregress(x1,x2) #predicting values of x2 based on x1
z1_p = x1 - (nu_1.intercept + nu_1.slope*x2)
z2_p = x2 - (nu_2.intercept + nu_2.slope*x1)
r_1 = stats.linregress(z1_p,y)
r_2 = stats.linregress(z2_p,y)
yu_1_p = r_1.intercept + r_1.slope*z1_p
yu_2_p = r_2.intercept + r_2.slope*z2_p
Vu_1_p = sum((yu_1_p-loc_y)**2)/(N-1)
Vu_2_p = sum((yu_2_p-loc_y)**2)/(N-1)

#Correlated variations
Vc_1_p = Vt_1_p-Vu_1_p
Vc_2_p = Vt_2_p-Vu_2_p

In [104]:
Vt_1_p

16.79080615607169

In [105]:
Vt_2_p

19.341926186454646

In [106]:
Vu_1_p

2.0430071739208993

In [107]:
Vu_2_p

4.594127204303066

In [108]:
Vc_1_p

14.74779898215079

In [109]:
Vc_2_p

14.74779898215158

### 2. Nonlinear model: $y = \frac{x_1x_2}{x_3}$

In [ ]:
lower, upper = [1,1,1], [10,10,10]
